In [146]:
#!pip install qdrant-client

In [147]:
#! pip install clip

In [3]:
import os
import torch
import skimage
import requests
import numpy as np
import pandas as pd
from PIL import Image
from io import BytesIO
import IPython.display
import matplotlib.pyplot as plt
from collections import OrderedDict
from transformers import CLIPProcessor, CLIPModel, CLIPTokenizer
import json
from tqdm import tqdm
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import clip
import zipfile

In [5]:
from qdrant_client import QdrantClient

# Connect to qdrant client
try:
    client = QdrantClient(
        url="https://c487cd09-8b2d-4c48-95cb-0bf3209ad30a.us-east4-0.gcp.cloud.qdrant.io:6333",
        api_key="wy51l0VFwIIaR_wu_pOaDEATBzLnCCxSrDG_uMFm1kSOozNByXKwlQ",
        timeout=None
    )
    print("client created successfully")

except QuadrantAPIError as e:
    # Handle API errors, if any
    print("Error creating Quadrant client:", e)
except Exception as e:
    # Handle any other unexpected errors
    print("An unexpected error occurred:", e)

client created successfully


In [63]:
from qdrant_client.http.models import VectorParams, Distance

# Create a new collection
client.recreate_collection(
   collection_name="selected_articles",
   vectors_config={
       "image": VectorParams(
           size=512,
           distance=Distance.COSINE,
       ),
   }
)

True

In [6]:
# Get collection info
client.get_collection(collection_name="selected_articles")

CollectionInfo(status=<CollectionStatus.GREEN: 'green'>, optimizer_status=<OptimizersStatusOneOf.OK: 'ok'>, vectors_count=27284, indexed_vectors_count=19930, points_count=27284, segments_count=2, config=CollectionConfig(params=CollectionParams(vectors={'image': VectorParams(size=512, distance=<Distance.COSINE: 'Cosine'>, hnsw_config=None, quantization_config=None, on_disk=None)}, shard_number=1, sharding_method=None, replication_factor=1, write_consistency_factor=1, read_fan_out_factor=None, on_disk_payload=True, sparse_vectors=None), hnsw_config=HnswConfig(m=16, ef_construct=100, full_scan_threshold=10000, max_indexing_threads=0, on_disk=False, payload_m=None), optimizer_config=OptimizersConfig(deleted_threshold=0.2, vacuum_min_vector_number=1000, default_segment_number=0, max_segment_size=None, memmap_threshold=None, indexing_threshold=20000, flush_interval_sec=5, max_optimization_threads=1), wal_config=WalConfig(wal_capacity_mb=32, wal_segments_ahead=0), quantization_config=None), p

In [7]:
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


In [8]:
articles=pd.read_csv("/content/drive/My Drive/selected_articles.csv",index_col=0)

In [9]:
# Adding the missing 0 to the article id and converting it to string as per the original dataset
articles['article_id'] = articles['article_id'].astype(str)
articles['article_id']=["0"+x for x in articles['article_id']]

In [10]:
df = articles[['article_id','caption']]

In [11]:
z1 = zipfile.ZipFile("/content/drive/My Drive/selected_images_1.zip")
files_in_zip_1 = z1.namelist()

In [12]:
id_to_path_mapping = {file.split('/')[1].split('.')[0]: file for file in files_in_zip_1}
df['image_path'] = df['article_id'].map(id_to_path_mapping)

del id_to_path_mapping

<ipython-input-12-6dfaf1e8d921>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['image_path'] = df['article_id'].map(id_to_path_mapping)


In [13]:
df = df.dropna()

In [14]:
df.reset_index(inplace=True)

In [15]:
df=df.drop(['index'],axis=1)

In [16]:
df

,article_id,caption,image_path
0,0108775015,"womens Solid Black Vest top, Jersey top with n...",selected_images_1/0108775015.jpg
1,0108775044,"womens Solid White Vest top, Jersey top with n...",selected_images_1/0108775044.jpg
2,0108775051,"womens Stripe Off White Vest top, Jersey top w...",selected_images_1/0108775051.jpg
3,0111586001,"womens Solid Black Leggings, Tights with built...",selected_images_1/0111586001.jpg
4,0116379047,"womens Solid Dark Blue Top, Fitted top in soft...",selected_images_1/0116379047.jpg
...,...,...,...
27279,0704630002,"womens Solid Light Orange Sneakers, Trainers w...",selected_images_1/0704630002.jpg
27280,0704630003,"womens Colour blocking White Sneakers, Trainer...",selected_images_1/0704630003.jpg
27281,0704630004,"womens Colour blocking Black Sneakers, Trainer...",selected_images_1/0704630004.jpg
27282,0704630005,"womens Colour blocking Orange Sneakers, Traine...",selected_images_1/0704630005.jpg


In [17]:
# Function to get CLIP model, processor and tokenizer
def get_model_info(model_ID, device):

    model = CLIPModel.from_pretrained(model_ID).to(device)
    processor = CLIPProcessor.from_pretrained(model_ID)
    tokenizer = CLIPTokenizer.from_pretrained(model_ID)

    return model, processor, tokenizer

In [18]:
# Function to convert model's parameters to FP32 format
def convert_models_to_fp32(model):

    for p in model.parameters():
        p.data = p.data.float()
        p.grad.data = p.grad.data.float()

In [19]:
# Function to extract images from the given image path and store them as PIL objects
def get_image(image_path):
    if image_path in z1.namelist():
        image = [Image.open(BytesIO(z1.read(image_path))).convert("RGB")]

    return image

In [67]:
# Defining a class to prepare dataset for batch processing
class image_title_dataset():
    def __init__(self, list_id,list_txt):

        self.ids = list_id
        self.title  = list_txt

    def __len__(self):
        return len(self.title)

    def __getitem__(self, idx):

        return self.ids[idx], self.title[idx]

In [21]:
device = "cuda" if torch.cuda.is_available() else "cpu"
model_ID = "openai/clip-vit-base-patch32"

model, processor, tokenizer = get_model_info(model_ID, device)

if device == "cpu":
    model.float()

optimizer = torch.optim.Adam(model.parameters(), lr=5e-5,betas=(0.9,0.98),eps=1e-6,weight_decay=0.2) # the lr is smaller, more safe for fine tuning to new dataset

loss_img = nn.CrossEntropyLoss()
loss_txt = nn.CrossEntropyLoss()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/4.19k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/605M [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/316 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/568 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/862k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/525k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.22M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/389 [00:00<?, ?B/s]

In [22]:
loss = nn.CrossEntropyLoss()

In [108]:
from qdrant_client.http import models

# Get image embeddings and create records to be added into the collection
records=[]
for index, row in df.iterrows():
  # if index==1000:
  #   break
  image_encode = processor(text=None, images=get_image(row['image_path']), return_tensors="pt", padding=True)['pixel_values']
  image_encode=image_encode.to(device)
  #image_encode=image_encode.view(-1).numpy()
  img_embds = model.get_image_features(image_encode)
  img_embds = img_embds.cpu().detach().numpy().tolist()[0]
  records.append(models.Record(id=index, vector=img_embds, payload={"caption": row['caption']}))

In [142]:
# Retrieve image embeddings from the collection and normalize
def get_image_embeddings(id):
  point = client.retrieve(collection_name="selected_articles", ids=[id], with_vectors=True)[0].vector['image']
  point = torch.tensor(point, requires_grad=False).view(1,512).to(device)
  #point= point.view(1,512).to(device)/ point.view(1, 512).to(device).norm(p=2, dim=-1, keepdim=True)
  point= point/ point.norm(p=2, dim=-1, keepdim=True)
  return point

In [143]:
# Get text embeddings and normalize
def get_text_embeddings(text):
  point = tokenizer(text, return_tensors="pt").to(device)
  point = model.get_text_features(**point)
  #point = point.cpu().detach().numpy().tolist()[0]
  point = torch.tensor(point, requires_grad=False).view(1,512).to(device)
  point = point / point.norm(p=2,dim=-1, keepdim=True)

  return point

In [76]:
# Delete points from collection
# client.delete(
#     collection_name="selected_articles",
#     points_selector=models.PointIdsList(
#         points=ids,
#     ),
# )


UpdateResult(operation_id=8, status=<UpdateStatus.COMPLETED: 'completed'>)

In [68]:
# Train-validatin-test split
X_train, X_test, y_train, y_test = train_test_split(df.index, df['caption'], test_size=0.2, random_state=1)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=1)

train_data = pd.DataFrame({'id':X_train.values, 'caption': y_train.values})
val_data = pd.DataFrame({'id':X_val.values, 'caption': y_val.values})
test_data = pd.DataFrame({'id':X_test.values, 'caption': y_test.values})

In [115]:
# Create data loaders

list_id_train = []
list_txt_train = []
list_id_val = []
list_txt_val = []

for index,item in train_data.iterrows():
    # Taking first 2000 image caption pairs from train data
    if index==100:
        break
    id = item['id']
    caption = item['caption'][:77]
    list_id_train.append(id)
    list_txt_train.append(caption)

for index,item in val_data.iterrows():
  # Taking first 1000 image caption pairs from validation data
    if index==100:
        break
    id = item['id']
    caption = item['caption'][:77]
    list_id_val.append(id)
    list_txt_val.append(caption)


train_dataset = image_title_dataset(list_id_train, list_txt_train)
val_dataset = image_title_dataset(list_id_val, list_txt_val)

train_dataloader = DataLoader(train_dataset, batch_size=10, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=10, shuffle=True)

In [113]:
logit_scale = model.logit_scale.exp()

In [145]:
# Finetuning
num_epochs = 10
train_loss_per_epoch=[]
val_loss_per_epoch=[]


for epoch in range(num_epochs):

    # Finetuning on train data
    train_losses=0

    with tqdm(total=len(train_dataloader), desc=f'Epoch {epoch + 1} - Training') as pbar_train:

        # Processing for each train batch
        for ids, texts in train_dataloader:
            optimizer.zero_grad()

            # Load images from image paths

            image_list = [get_image_embeddings(id.item()) for id in ids]
            text_list = [get_text_embeddings(text) for text in texts]
            #image_list=[get_image(img) for img in images]

            # Encode the images and texts using CLIP's processor
            # inputs = processor(text=texts, images=image_list, return_tensors="pt", padding=True)
            # inputs=inputs.to(device)

            # # Get image and text embeddings using CLIP model
            # outputs = model(**inputs)

            # # Get similarity scores between the image-text embeddings
            # logits_per_image, logits_per_text = outputs.logits_per_image, outputs.logits_per_text

            # Compute loss
            # clip_cos_sim = [torch.mm(x, y.T)[0] for x,y in zip(text_list, image_list)]
            # probabilities = F.softmax(torch.cat(clip_cos_sim, dim=0), dim=0)
            # probabilities=torch.tensor(probabilities,requires_grad=True)

            clip_cos_sim = [torch.nn.functional.cosine_similarity(x, y) * logit_scale for x,y in zip(text_list, image_list)]
            # ground_truth = torch.arange(len(images),dtype=torch.long,device=device)
            # total_loss = (loss_img(logits_per_image,ground_truth) + loss_txt(logits_per_text,ground_truth))/2

            ground_truth = torch.arange(len(ids),dtype=torch.float32,device=device)#, requires_grad=True)
            total_loss = loss(torch.cat(clip_cos_sim, dim=0),ground_truth)

            # Perform backward pass
            total_loss.backward(retain_graph=True)
            if device == "cpu":
                optimizer.step()

            else :
                convert_models_to_fp32(model)
                optimizer.step()
                clip.model.convert_weights(model)

            train_losses += total_loss.item()
            pbar_train.update(1)

        # Get final train loss for the epoch
        final_train_loss=train_losses/len(train_dataloader)


    # # Evaluating on validation data - same as above but without any parameter updates
    # val_losses=0

    # with torch.no_grad():
    #     with tqdm(total=len(val_dataloader), desc=f'Epoch {epoch + 1} - Validation') as pbar_val:
    #         for images, texts in val_dataloader:

    #             image_list=[get_image(img) for img in images]
    #             inputs = processor(text=texts, images=image_list, return_tensors="pt", padding=True)

    #             inputs=inputs.to(device)
    #             outputs = model(**inputs)
    #             logits_per_image, logits_per_text = outputs.logits_per_image, outputs.logits_per_text

    #             ground_truth = torch.arange(len(images),dtype=torch.long,device=device)
    #             total_loss = (loss_img(logits_per_image,ground_truth) + loss_txt(logits_per_text,ground_truth))/2

    #             val_losses += total_loss.item()
    #             pbar_val.update(1)

    #         # Get final validation loss for the epoch
    #         final_val_loss=val_losses/len(val_dataloader)

    # Print the losses
    print(f'Epoch {epoch + 1} - Train Loss: {final_train_loss:.4f}')#, Validation Loss: {final_val_loss:.4f}')
    train_loss_per_epoch.append(final_train_loss)
    #val_loss_per_epoch.append(final_val_loss)

Epoch 1 - Training:   0%|          | 0/10 [00:00<?, ?it/s]<ipython-input-143-b6663d812f08>:5: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  point = torch.tensor(point, requires_grad=False).view(1,512).to(device)
Epoch 1 - Training:   0%|          | 0/10 [00:01<?, ?it/s]


RuntimeError: Trying to backward through the graph a second time (or directly access saved tensors after they have already been freed). Saved intermediate values of the graph are freed when you call .backward() or autograd.grad(). Specify retain_graph=True if you need to backward through the graph a second time or if you need to access saved tensors after calling backward.